The reference for this notebook is [here](https://medium.com/aimpact-all-things-ai/prompt-like-a-pro-using-dspy-a-guide-to-build-a-better-local-rag-model-using-dspy-qdrant-and-d8011a3942d9#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6Ijg3YmJlMDgxNWIwNjRlNmQ0NDljYWM5OTlmMGU1MGU3MmEzZTQzNzQiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJhdWQiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJzdWIiOiIxMTQzMDg4NjQ0NjgxMzkxMjUxNjAiLCJlbWFpbCI6ImRzbWl0aC5zbWl0aDg2NEBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwibmJmIjoxNzIwMjkwNDM1LCJuYW1lIjoiRGFuaWVsIFNtaXRoIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FDZzhvY0tCWm9sTU8tUkk4VzYySEhTUl9KMFdIS3lFUkdoR1k0Njlsa0FNMEZTSmdSbFhMdz1zOTYtYyIsImdpdmVuX25hbWUiOiJEYW5pZWwiLCJmYW1pbHlfbmFtZSI6IlNtaXRoIiwiaWF0IjoxNzIwMjkwNzM1LCJleHAiOjE3MjAyOTQzMzUsImp0aSI6IjQ2MzJkYTY1Yjc0OWY5MWUxOGJhZTVjMzhkN2YzY2I0ZmJiNzRjYTIifQ.aP4dZS1Pej36uxs6rJo1NHDlRPKSZV26gJWH5UTWVuVtWJ46fQ2LyvaCGG2my4zQnzu_2RB2V3kPREFz8tP8HwrLkwEa0oWIz-uko6le7F67xBRdTWylpnaqPJtGJeZua9QKqwIZTAe9kZPiN8wIx-v7QwriOGqM7AMi-gPBRXEMCH6ToMoCdDp1XH0a-mLtK8FIGUK8V2EFUsOxRXv2vJmo6NnuwxxHtnH36hphhS6WRToVBxTI7Lm85Xr0IJtoDuhVX8VXlYhVMk8lT3srQLv15TiykFFKZFwvCfmr9WzhOmKiQBlZTIIl8ff2_MMVa7Qk7YO3fFuDv6E2WOSYOQ)

In [ ]:
import dspy

## Load the dataset

In [ ]:
classes = ["pulmonary edema", "consolidation", "pleural effusion", "pneumothorax", "cardiomegaly"]

reports = [
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Chest pain. Comparison: Chest radiograph from XXXX, XXXX. Findings: The cardiac silhouette is borderline enlarged. Otherwise, there is no focal opacity. Mediastinal contours are within normal limits. There is no large pleural effusion. No pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Borderline enlargement of the cardiac silhouette without acute pulmonary disease. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Shortness of breath. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is evidence of bilateral pulmonary edema. The cardiac silhouette is normal. No pleural effusion or pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Bilateral pulmonary edema. No evidence of pleural effusion or pneumothorax. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Cough and fever. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is consolidation in the right lower lobe. The cardiac silhouette is normal. No pleural effusion or pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Right lower lobe consolidation. No pleural effusion or pneumothorax. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Chest pain. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is a small left pleural effusion. The cardiac silhouette is normal. No pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Small left pleural effusion. No pneumothorax. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Trauma. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is a right-sided pneumothorax. The cardiac silhouette is normal. No pleural effusion. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Right-sided pneumothorax. No pleural effusion. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Shortness of breath and leg swelling. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is moderate pulmonary edema and bilateral pleural effusion. The cardiac silhouette is enlarged. No pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Moderate pulmonary edema and bilateral pleural effusion. Cardiomegaly. No pneumothorax. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Fever and cough. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is a consolidation in the left upper lobe. The cardiac silhouette is normal. No pleural effusion or pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Left upper lobe consolidation. No pleural effusion or pneumothorax. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Routine check-up. Comparison: Chest radiograph from XXXX, XXXX. Findings: The cardiac silhouette is normal. No focal opacity. Mediastinal contours are within normal limits. There is no pleural effusion or pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Normal chest radiograph. No abnormalities detected. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Dyspnea. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is mild cardiomegaly. Bilateral pleural effusions are present. No evidence of pneumothorax. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Mild cardiomegaly with bilateral pleural effusions. No pneumothorax. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """,
    """
    RADIOLOGY REPORT

    Exam
    PA and lateral chest radiograph (2 views) (2 images) Date: XXXX, XXXX at XXXX hours Indication: Trauma. Comparison: Chest radiograph from XXXX, XXXX. Findings: There is a left-sided pneumothorax. The cardiac silhouette is normal. No pleural effusion. Transcribed by - PSCB Transcription Date - XXXX

    IMPRESSION
    Left-sided pneumothorax. No pleural effusion. DICTATED BY : Dr. XXXX XXXX XXXX XXXX XXXX ELECTRONICALLY SIGNED XXXX. XXXX XXXX XXXX XXXX XXXX TRANSCRIBED XXXX 11 XXXX XXXX  RADRES XXXX

    SIGNATURE
    XXXX
    """
]

# Ground Truth Labels for each report
ground_truth = [
    ["cardiomegaly"],
    ["pulmonary edema"],
    ["consolidation"],
    ["pleural effusion"],
    ["pneumothorax"],
    ["pulmonary_edema", "pleural effusion", "cardiomegaly"],
    ["consolidation"],
    [],
    ["cardiomegaly", "pleural effusion"],
    ["pneumothorax"]
]

In [ ]:
from dspy.retrieve.qdrant_rm import QdrantRM
from qdrant_client import QdrantClient

In [ ]:
qdrant_client = QdrantClient(":memory:")  # In-memory load

In [ ]:
docs = classes
ids = list(range(0,len(docs)))

In [ ]:
qdrant_client.add(
    collection_name="rads",
    documents=docs,
    ids=ids
    )

## Define the retriever

In [ ]:
qdrant_retriever_model = QdrantRM("rads", qdrant_client, k=4)

In [ ]:
dspy.settings.configure(rm=qdrant_retriever_model)

In [ ]:
def get_top_passages(question):
    retrieve = dspy.Retrieve(k=4)
    topK_passages = retrieve(question, k=4).passages
    print(topK_passages)
    print(f"Top {retrieve.k} passages for question: {question} \n", '-' * 30, '\n')
    for idx, passage in enumerate(topK_passages):
        print(f'{idx+1}]', passage, '\n')

In [ ]:
dev_example = reports[1]

get_top_passages(dev_example)

## Initialize Llama3 Model Using DSPy-Ollama Integration

In [ ]:
ollama_model = dspy.OllamaLocal(model="llama3",model_type='text',
                                max_tokens=350,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)

In [ ]:
ollama_model("tell me about interstellar's plot")

In [ ]:
dspy.settings.configure(lm=ollama_model, rm=qdrant_retriever_model)

## Define Signatures for Input and Output

### TODO 
- Add dspy assertions to enforce the output to be a list of classes

In [ ]:
# class RetrieveLabelCandidates(dspy.Signature):
#     """Retrieve relevant label candidates for a given text."""
#     text = dspy.InputField()
#     label_candidates = dspy.OutputField(desc="List of potential labels for the given text")

# class ClassifyText(dspy.Signature):
#     """Classify the text into multiple labels from the given candidates."""
#     text = dspy.InputField()
#     label_candidates = dspy.InputField(desc="List of possible labels for the text")
#     labels = dspy.OutputField(desc="List of applicable labels for the text")

class ClassifyText(dspy.Signature):
    """Classify the radiology into multiple labels from the given candidates. You should return the 
    extracted information as a single JSON string with a key for each candidate label and a value of
    1 if the report indicates the presence of the abnormality and 0 otherwise. There should be no 
    text or explanation, only the JSON. For example if there 
    were 3 candidates you could have the following output:

    {
        "label_1": 1,
        "label_2": 0,
        "label_3": 1
    }"""
    text = dspy.InputField()
    label_candidates = dspy.InputField(desc="List of candidate labels for the text")
    labels = dspy.OutputField(desc="Dictionary of candidate labels, 1 or 0, for the text")

In [ ]:
reports[1]

In [ ]:
ct = ClassifyText(text = reports[1], label_candidates = ','.join(classes), labels='')
# ct = ClassifyText(text = reports[1], label_candidates = ','.join(classes))
# Need to figure out how to use the retriever model to populate `label_candidates`

In [ ]:
print(ct.model_construct)

In [ ]:
# class GenerateAnswer(dspy.Signature):
#     """Answer questions with short factoid answers."""

#     context = dspy.InputField(desc="may contain relevant facts or answer keywords")
#     question = dspy.InputField()
#     answer = dspy.OutputField(desc="an answer between 1 to 10 words")

In [ ]:
# ga = GenerateAnswer(context="My name is sachin and I like writing blogs", question="What is my name?", answer="Sachin")
# print(ga.model_construct)

## Create a DSPy Module

In [ ]:
# class RAGMultiLabelClassifier(dspy.Module):
#     def __init__(self, num_candidates=5):
#         super().__init__()
#         self.retrieve = dspy.Retrieve(k=num_candidates)
#         self.classify = dspy.Predict(ClassifyText)
#         self.num_candidates = num_candidates

#     def forward(self, text):
#         # Retrieve relevant documents
#         # retrieved_docs = self.retrieve(text).passages
#         retrieved_docs = ','.join(self.retrieve(text).passages)

#         # Generate label candidates
#         # candidate_result = self.generate_candidates(text=text)
#         # label_candidates = candidate_result.label_candidates[:self.num_candidates]

#         # Classify text
#         classification_result = self.classify(text=text, label_candidates=retrieved_docs)
#         return classification_result.labels


class RAGMultiLabelClassifier(dspy.Module):
    def __init__(self, num_candidates=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_candidates)
        self.classify = dspy.Predict(ClassifyText)

    def forward(self, text):
        retrieved_docs = ','.join(self.retrieve(text).passages)
        classification_result = self.classify(text=text, label_candidates=retrieved_docs)
        return classification_result.labels

In [ ]:
classifier = RAGMultiLabelClassifier()
result = classifier(reports[1])
print(result)

In [ ]:
ollama_model.inspect_history(n=1)